### MountainCar 회귀신경망

In [1]:
import gym

env = gym.make('MountainCarContinuous-v0') #연속적인 행동 공간, 실수형

# x위치: -1.2 ~ 0.6
# 속도: -0.07~0.07
# 선택 옵션 3가지 : 왼쪽으로 이동, 정지, 오른쪽으로 이동
print(env.observation_space)
print(env.observation_space.low)
print(env.observation_space.high, '\\n')
print(env.action_space, '\\n')
print(env._max_episode_steps)

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
[-1.2  -0.07]
[0.6  0.07] \n
Box(-1.0, 1.0, (1,), float32) \n
999


In [2]:
# 랜덤 에이전트
# 보상 : 행동의 제곱에 0.1을 곱한 값의 음수
# 깃발에 도달하면 +100
# 움직이지 않으면 보상 0, 큰 힘으로 움직일수록 음의 보상을 더 많이 받음

env.reset()

score = 0
step = 0

for i in range(200):
  action = env.action_space.sample()
  obs, reward, done, info = env.step(action)[:4]
  previous_obs = obs
  score += reward
  step += 1

  if done:
    break
  
print(score, step)

-6.858460676322903 200


c:\Python3.9.13\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [3]:
env.reset()

score = 0
step = 0

while True:
  action = env.action_space.sample()
  obs, reward, done, info = env.step(action)[:4]
  previous_obs = obs
  score += reward
  step += 1

  if done:
    break

print(score, step)

-175.03660254896644 8108


In [4]:
# 성공적인 에피소드 저장
# 스텝당 -1, x 좌표로 -0.2 이상일 때 +1 보상

import numpy as np

scores = []
training_data = []
accepted_scores = []

required_score = -198

for i in range(10000):
  if i % 100 == 0:
    print(i,end=' ')

  env.reset()

  score = 0

  game_memory = []
  previous_obs = []
  
  for i in range(200):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)[:4]

    if len(previous_obs) > 0:
      game_memory.append([previous_obs, action])
    previous_obs = obs

    if obs[0] > -0.2:
      reward = 1
    else:
      reward = -1

    score += reward

    if done:
      break

  scores.append(score)

  if score > required_score:
    accepted_scores.append(score)
    for data in game_memory:
      training_data.append(data)

scores = np.array(scores)

print(scores.mean())
print(scores)
print(accepted_scores)

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 -199.8702
[-200 -200 -200 ... -200 -200 -200]
[-192, -188, -174, -180, -176, -192, -182, -180, -188, -182, -184, -178, -188, -178, -188, -174, -194, -152, -196, -172, -194, -186, -174, -172, -192, -174, -176, -180, -176, -182, -196, -194, -184, -184, -172, -174, -180, -174, -190, -174, -196, -178, -184, -182, -146, -166, -184, -170, -190, -182, -166, -176, -174, -190, -182, -184, -188, -196, -188, -178, -168, -192, -186, -178, -178, -188, -166, -184]


In [5]:
train_X = np.array([i[0] for i in training_data]).reshape(-1, 2)
train_Y = np.array([i[1] for i in training_data]).reshape(-1, 1)

print(train_X.shape)
print(train_Y.shape)
print(train_X)
print(train_Y)

(13532, 2)
(13532, 1)
[[-0.47763282 -0.00152474]
 [-0.47907153 -0.00143872]
 [-0.48188272 -0.00281119]
 ...
 [-0.2233712  -0.01063838]
 [-0.23517519 -0.011804  ]
 [-0.2500572  -0.01488199]]
[[ 0.28644508]
 [-0.69300556]
 [ 0.12974678]
 ...
 [ 0.5291701 ]
 [-0.7832091 ]
 [ 0.6974685 ]]


In [6]:
import tensorflow as tf

model = tf.keras.Sequential([
              tf.keras.layers.Dense(128, input_shape=(2,), activation='elu'),
              tf.keras.layers.Dense(32, activation='elu'),
              tf.keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer=tf.optimizers.Adam(), loss='mse')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               384       
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4545 (17.75 KB)
Trainable params: 4545 (17.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
history = model.fit(train_X, train_Y, epochs=10, validation_split=0.25)

Epoch 1/10
318/318 [==============================] - 1s 2ms/step - loss: 0.3435 - val_loss: 0.3439
Epoch 2/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3388 - val_loss: 0.3432
Epoch 3/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3374 - val_loss: 0.3410
Epoch 4/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3377 - val_loss: 0.3444
Epoch 5/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3374 - val_loss: 0.3449
Epoch 6/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3376 - val_loss: 0.3409
Epoch 7/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3376 - val_loss: 0.3420
Epoch 8/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3379 - val_loss: 0.3415
Epoch 9/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3377 - val_loss: 0.3436
Epoch 10/10
318/318 [==============================] - 0s 1ms/step - loss: 0.3373 - val_loss: 0.3416

In [8]:
#회귀신경망으로 테스트

from gymnasium.experimental.wrappers import RecordVideoV0
from IPython.display import Video

scores = []
steps = []
actions = []

# 환경 생성하기(미로 탐색 게임)
env = gym.make("MountainCarContinuous-v0", render_mode='rgb_array')
env = RecordVideoV0(env, "video", name_prefix="mount-regress", disable_logger=True,episode_trigger=lambda x: x % 2 == 0 )

for i in range(3):
  score = 0
  step = 0
  cnt=0

  previous_obs = []

  env.reset()

  while True:
    cnt+=1

    if cnt%100==0:
      print(cnt,end=' ')

    if len(previous_obs) == 0:
      action = env.action_space.sample()

    else:
      action = model.predict(np.expand_dims(previous_obs, axis=0),verbose=None)[0]
      actions.append(action)

    obs, reward, done, info = env.step(action)[:4]
    previous_obs = obs
    score += reward
    step += 1

    if done:
        break

  scores.append(score)
  steps.append(step)

  print('\n',i, 'mean score: {}, mean step: {}'.format(np.mean(scores[-100:]), np.mean(steps[-100:])))

env.close()        

c:\Python3.9.13\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


100 200 300 400 500 600 700 800 
 0 mean score: 99.33608861270734, mean step: 803.0
100 200 300 400 500 600 700 800 900 
 1 mean score: 99.33245511049637, mean step: 892.0
100 200 300 400 500 600 700 
 2 mean score: 99.33862282074814, mean step: 837.6666666666666


In [9]:
Video("video/mount-regress-episode-2.mp4", embed=True)